In [ ]:
# 线性回归
from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import fetch_california_housing as fch
import pandas as pd


In [ ]:
housevalue = fch()

In [ ]:
housevalue.data # 看不出什么
X = pd.DataFrame(housevalue.data) # 放入dataframe中便于查看
y = housevalue.target
X.shape #(20640, 8)
y.shape # (20640,)
y.min()
y.max() # 不是房价本身，而是房价的评分
X.columns = housevalue.feature_names

In [ ]:
X.columns
"""
MedInc: 该街区住户的收入中位数
HouseAge: 房屋使用年代的额中位数
AveRooms: 平均房屋数量
AveBedrms: 平均卧室数量
Population: 街区人口
AveOccup: 平均入住率
Latitude: 街区的纬度
Longitude:街区的经度
"""
X.head()

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=420)
x_valid.head()
# 恢复索引
for i in [x_train, x_valid]:
    i.index = range(i.shape[0])
x_train.shape
# 进行数据标准化，要先分训练集和测试集，再做别的东西，先用训练集训练标准化的类，然后用训练好的类再分别转化训练集和测试集
# 建模
reg = LR().fit(x_train, y_train)
yhat = reg.predict(x_valid)
yhat
yhat.min()
yhat.max() # 比原来的真实值还大了挺多

reg.coef_ #w, 系数向量
[*zip(x_train.columns, reg.coef_)]
# 截距
reg.intercept_

In [ ]:
# 看均方误差
from sklearn.metrics import mean_squared_error as MSE
MSE(yhat, y_valid) #0.5
y_valid.mean() # 2.0, 相比平均值误差已经达到了20%多
y.max()
y.min()
# 用交叉验证进行求解均方误差， 评估指标错了
# cross_val_score(reg, X, y, cv=10, scoring="mean_squared_error")
# 为什么报错

import sklearn
sorted(sklearn.metrics.SCORERS.keys()) # 可用的模型评估指标
cross_val_score(reg,X, y, cv=10, scoring="neg_mean_squared_error")
# 均方误差为负，实际为真，是相反数
(cross_val_score(reg, X, y, cv=10, scoring="neg_mean_squared_error") * (-1)).mean()
# R^2
from sklearn.metrics import r2_score
r2_score(yhat, y_valid)
r2 = reg.score(x_valid, y_valid)
r2
# 两种调用的结果R^2是不一样的？
r2_score(y_valid, yhat)
# 这样就一样了，所以输入的顺序很重要
#或者可以指定参数，这样就不会顺序出错了
r2_score(y_true=y_valid, y_pred=yhat)
cross_val_score(reg,X,y,cv=10,scoring="r2").mean()


# 画个图看看
import matplotlib.pyplot as plt
sorted(y_valid)
# 排序是为了不像个散点图
plt.figure()
plt.plot(range(len(y_valid)), sorted(y_valid), c="black", label="Data")
plt.plot(range(len(yhat)), sorted(yhat), c="red", label="predict")
plt.legend()
plt.show()

# 所以R^2才是最重要的指标


In [ ]:
import numpy as np
rng = np.random.RandomState(42)
X = rng.randn(100, 80)
y = rng.randn(100)
cross_val_score(LR(), X, y, cv=5, scoring="r2")# 得到5个交叉验证的结果
# 出现负数的R^2


In [ ]:
# 岭回归
# 核心参数就是系数a。如果我们的数据集在岭回归中的正则化参数各种取值下没有明显上升，则说明没有多重共线性
#，顶多是特征之间存在一些相关性
# 反之证明有多重共线性
# 重新测试
from sklearn.linear_model import Ridge, LinearRegression, Lasso
import matplotlib.pyplot as plt
housevalue = fch()
X = pd.DataFrame(housevalue.data)
y = housevalue.target
X.columns = ["住房收入中位数", "使用年代中位数", "平均房屋数目", "平均卧室数目", "街区人口", "平均入住率", "街区的维度", "街区的经度"]
X.head()

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=420)
for i in [x_train, x_valid]:
    i.index = range(i.shape[0])

reg = Ridge(alpha=1).fit(x_train, y_train)
reg.score(x_valid, y_valid) # 应该不是多重共线性的问题
# 下面验证交叉验证下，岭回归的结果如何变动
alpharange = np.arange(1, 1001, 100)
ridge, lr = [], []
for alpha in alpharange:
    reg = Ridge(alpha=alpha)
    linear = LinearRegression()
    regs = cross_val_score(reg, X, y, cv=5, scoring="r2").mean()
    linears = cross_val_score(linear, X, y, cv=5, scoring="r2").mean()
    ridge.append(regs)
    lr.append(linears)
plt.figure()
plt.plot(alpharange, ridge, color="red", label="Ridge")
plt.plot(alpharange, lr, color="orange", label="LR")
plt.title("Mean")
plt.legend()
plt.show()
# 证明只有一些共线性
# 再花0 200 之间
# 求最优alpha
# 使用岭迹图进行选择，已经被淘汰# 直接交叉验证好了
# 交叉验证进行岭回归
#RidgeCV(
    #alphas:需要正则化的参数的元组
    # scoring R^2
    #store_cv_values是否保存每次结果
    # cv :交叉验证模式，默认为cv留一交叉验证
#)
# cv只有是None才会保存cv的结果


In [ ]:
# lasso
from sklearn.linear_model import Ridge, LinearRegression, Lasso
import matplotlib.pyplot as plt
housevalue = fch()
X = pd.DataFrame(housevalue.data)
y = housevalue.target
X.columns = ["住房收入中位数", "使用年代中位数", "平均房屋数目", "平均卧室数目", "街区人口", "平均入住率", "街区的维度", "街区的经度"]
X.head()

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=420)
for i in [x_train, x_valid]:
    i.index = range(i.shape[0])

reg = LinearRegression().fit(x_train, y_train)
(reg.coef_ * 100).tolist() # coef_ 是系数w
Ridge_ = Ridge(alpha=0).fit(x_train,y_train)
(Ridge_.coef_ * 100).tolist() # 和线性回归基本一样
lasso_ = Lasso(alpha=0).fit(x_train, y_train)
(lasso_.coef_ * 100).tolist() # lasso对alpha的变动影响非常大, 而且可以把w压缩到0

#lassoCV 因为lasso对正则化系数非常的敏感，所以设定了正则化路径的概念
# 正则化路径
# 建模应用的是军方误差
#alphas_
#mse_path 细节结果


In [ ]:
from sklearn.linear_model import LassoCV
# 自己建立lasso进行alpha选择的范围
alpharange = np.logspace(-10, -2, 200,base=10)
# 从10 的－10次到10 的－2次取出200个数
alpharange.shape

In [ ]:
lasso_ = LassoCV(alphas=alpharange, cv=5).fit(x_train, y_train)
lasso_.alpha_
lasso_.mse_path_.shape # 返回了每个alpha下的五折交叉验证结果(200, 5)
lasso_.mse_path_.mean(axis=1)
# 岭回归中我们是axis=0
# 岭回归是留一验证，因此我们的交叉验证结果返回的是，每一个样本下再每个alpha下的交叉验证结果
# 而在这里，我们返回的是再每一个alpha下每一折的交叉验证的结果
#因此我们要求每个alpha下的交叉验证均值，就是axis=1 跨列求均值
lasso_.coef_
# 最佳正则化洗漱下的系数结果
# 贝叶斯信息准则，艾凯克信息准则。来做模型选择，同时还可以利用坐标下降，最小角度回归对lasso进行计算，算是走的最远的模型了
# 岭回归和lasso不是为了提升性能而是解决多重共线性而出现的，接下来是多项式回归，专门为了提升性能而设置的算法



In [ ]:
# 非线性问题： 多项式回归
# 两个的线性关系，在坐标系上是直线，在坐标系上是曲线，是非线性的
# 分段的不行，必须 一条 直线 是线性
# 线性回归 线性分类 都是最后是个直线
# 多元线性回归是线性的，自变量都是一次项
# 测试一下各个模型的算法
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
rnd = np.random.RandomState(42)# 设置随机数种子
X = rnd.uniform(-3, 3, size=100) # 从设置的两个整数中取出size个随机数,无顺序且无规律
# 生成y的思路，先使用numpy中的函数生成一个sin函数图像，然后再人为添加噪声
y = np.sin(X) + rnd.normal(size=len(X)) / 3 # normal 生成size个服从正太分布的随机数


In [ ]:
# 使用散点图
plt.scatter(X, y, marker="o", c="k", s=20)
plt.show() # 非线性
X.shape



In [ ]:
X = X.reshape(-1, 1) # 变成二维数组
LinearR =  LinearRegression().fit(X, y)
TreeR = DecisionTreeRegressor(random_state=0).fit(X, y)
# 放置画布,,画布和绘图要在同一个cell 中
fig, ax1 = plt.subplots(1)
# 创建测试数据，一系列分布再横坐标上的点
line = np.linspace(-3,3, 1000, endpoint=False).reshape(-1, 1)
# 将测试数据集带入predict，获得模型的拟合效果并进行绘制
ax1.plot(line, LinearR.predict(line), linewidth=2, color="green", label="linear regression")
ax1.plot(line, TreeR.predict(line), linewidth=2, color="red", label="decdision tree")
# 将原数据上的拟合绘制再图像上
ax1.plot(X[:, 0], y, "o", c="k")

ax1.legend(loc="best")
ax1.set_ylabel("regression output")
ax1.set_xlabel("input feature")
ax1.set_title("result before discretization")
plt.tight_layout()
plt.show()

# 非线性模型在线性上拟合比线性还好，甚至还会过拟合
# 线性拟合非线性精度很低, 但是可以用分箱（离散化）来进行处理，甚至比线性的还要好，线性的决策边界都是平行的直线，而非线性的模型的决策边界时曲线或者交叉的直线
# 回归问题，自变量上的最高次方1，分类问题，决策边界上最高此方1
# knn 朴素贝叶斯等等没有模型这种不区分线性和非线性
# 线性回归，逻辑回归，感知机等等为线性

In [ ]:
# 离散化（分箱)
from sklearn.preprocessing import KBinsDiscretizer

# 将数据分箱
enc = KBinsDiscretizer(n_bins=10 # 分几类
                        ,encode="onehot"
                        )


# 做哑变量离散化，形成一个（m， n_bins）每一列是一个分好的类别
# 对每一个样本而言，她包含的分类中他表示为1， 其余分类中她表示为0
X_binned = enc.fit_transform(X)
import pandas as pd
pd.DataFrame(X_binned.toarray()).head()


In [ ]:
# 进行画图
enc = KBinsDiscretizer(n_bins=10, encode="onehot")
X_binned = enc.fit_transform(X)
line_binned = enc.fit_transform(line)
# 将两张图绘制在一起，布置画布
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(10, 4))# 让两张图共享y轴的刻度
# 
ax1.plot(line, LinearR.predict(line), linewidth=2, color="green", label="linear regression")
ax1.plot(line, TreeR.predict(line), linewidth=2, color="red", label="decdision tree")
# 将原数据上的拟合绘制再图像上
ax1.plot(X[:, 0], y, "o", c="k")

ax1.legend(loc="best")
ax1.set_ylabel("regression output")
ax1.set_xlabel("input feature")
ax1.set_title("result before discretization")

# 使用分箱数据进行建模
LinearR_ = LinearRegression().fit(X_binned, y)
TreeR_ = DecisionTreeRegressor(random_state=0).fit(X_binned, y)

# 进行预测， 在图二中布置在分箱数据上进行预测的结果
ax2.plot(line # 横坐标
        , LinearR_.predict(line_binned)
        , linewidth=2
        , color="green"
        , linestyle="-"
        , label="linear regression"
        )
ax2.plot(line, TreeR_.predict(line_binned), linewidth=2, color="red", linestyle=":", label="decision tree")
# 绘制和箱一致的整线
ax2.vlines(enc.bin_edges_[0]#分箱上下限 x轴
            , *plt.gca().get_ylim()# y轴的上限和下线 plt.gca取出现有的图像,限制上下限 *是把两个数据分别用的意思
            , linewidth=1
            , alpha=.2)

ax2.plot(X[:, 0], y, "o", c="k")
ax2.set_ylabel("regression output")
ax2.set_xlabel("input feature")
ax2.set_title("result before discretization")

plt.tight_layout()
plt.show()
# 分箱子数量会影响结果

#如何选取箱子数呢，交叉验证取值



In [ ]:
from sklearn.model_selection import cross_val_score as CVS
import numpy as np

pred, score, var = [], [], []
binsrange = [2, 5, 10, 15, 20, 30]
for i in binsrange:
    enc = KBinsDiscretizer(n_bins=i, encode="onehot")
    X_binned = enc.fit_transform(X)
    line_binned = enc.fit_transform(line)
    linearR_ = LinearRegression()
    # 全数据集上的交叉验证
    cvresult = CVS(linearR_, X_binned, y, cv=5)
    score.append(cvresult.mean())
    var.append(cvresult.var())
    # 测试数据集上到额打分结果
    pred.append(linearR_.fit(X_binned, y).score(line_binned,np.sin(line)))

# 在画图



In [ ]:
# 多项式回归也可以用线性解决非线性
# 高纬呈现，低维解释
# 自变量的次数提升，就可以获得数据投影在高位里的结果
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
X = np.arange(1, 4).reshape(-1, 1)
poly = PolynomialFeatures(degree=2)

X_ = poly.fit_transform(X)
# [1, 2, 3]
# x0 x x^2 x^3
# [[1 1 1 1]
# [1 2 4 8 ]
# [1 3 9 27]]
# 这时候在高位中就是线性的了，可以利用线性拟合，然后每个给一个w权重
# 次数是n时
# [1, X X^2 X^3 X^4 X^n]
# 多项式公式
# include_bias 是生成x0吗
#.coef_ 是权重
## 查看截距interccept_
# 发现线性回归没有把x0当作截距
# 所以可以imclude_bias=False
#也可以关闭fit_intercept
#LinearRegression(fit_intercept=False).fit(xxx, y).coef_ 
#LinearRegression(fit_intercept=False).fit(xxx, y).intercept_ # 0


# 假如
X= np.arange(6).reshape(3, 2)
PolynomialFeatures(degree=2).fit_transform(X)
# 变成了x0 x1 x2 x1^2 x2^2 x1x2,六个维度
# 越高维度交互项越多，约有线性相关性
# 所以有参数interaction_only, 布尔值是否只产生交互项
# 求解多项式回归 处理非线性数据
from sklearn.preprocessing import PolynomialFeatures as PF
from sklearn.linear_model import LinearRegression
import numpy as np
rnd = np.random.RandomState(42)
X = rnd.uniform(-3, 3, size=100)
y = np.sin(X) + rnd.normal(size=len(X)) / 3
# 将x升维，准备好放入上课Learn中
X = X.reshape(-1, 1)
# 创建测试数据，
line = np.linspace(-3, 3, 1000, endpoint=False).reshape(-1, 1)
# 原始特征矩阵的拟合结果
LinearR = LinearRegression().fit(X, y)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# 对训练数据进行拟合
LinearR.score(X, y)
# 对测试数据的拟合
LinearR.score(line, np.sin(line))
# 多项式拟合，设定高次项
d = 5
# 进行高次转换
poly = PF(degree=d)
X_ = poly.fit_transform(X)
line_  = PF(degree=d).fit_transform(line)
# 训练数据的拟合

LinearR_ = LinearRegression().fit(X_, y)
LinearR_.score(X_, y)
# 测试数据的拟合
LinearR_.score(line_, np.sin(line))
# 直接起飞


In [ ]:
# 多项式回归的可解释性
from sklearn.linear_model import Ridge, LinearRegression, Lasso
import matplotlib.pyplot as plt
housevalue = fch()
X = pd.DataFrame(housevalue.data)
y = housevalue.target
X.columns = ["住房收入中位数", "使用年代中位数", "平均房屋数目", "平均卧室数目", "街区人口", "平均入住率", "街区的维度", "街区的经度"]
X.head()
y = housevalue.target
poly = PolynomialFeatures(degree=2).fit(X, y)
# 各种匹配的信息
poly.get_feature_names(X.columns)
X_ = poly.transform(X)
# 我们这时候依然可以直接建立模型，然后使用线性回归的coef——属性来查看什么特征对标签的影响最大
reg = LinearRegression().fit(X_, y)
coef = reg.coef_
[*zip(poly.get_feature_names(X.columns), reg.coef_)]
############################放到dataframe中进行排序
coeff = pd.DataFrame([poly.get_feature_names(X.columns), reg.coef_.tolist()]).T
coeff.columns = ["feature", "coef"]
coeff.sort_values(by="coef")
# 可以进行特征创造，比如路程等于速度乘时间


In [ ]:
# 用随机森林试一试
from time import time
time0 = time()
reg_ = LinearRegression().fit(X_, y)
print("R2:{}".format(reg_.score(X_, y)))
print("time:{}".format(time()-time0))


from sklearn.ensemble import RandomForestRegressor as RFR
time0 = time()
print("R2:{}".format(reg_.score(X_, y)))
print("time:{}".format(time()-time0))

# 多项式回归是线性还是非线性
# 自变量合并之后对于y是非线性的，但是就当作直接拿到几个变量， 这样看来就是线性的
#狭义认为自变量对于标签不能线性，广义认为参数对于标签不能线性，就是w不能相乘关系,所以定义不同

# 所有线性模型都可以多项式化





